In [ ]:
!pip install pint

In [ ]:
import numpy
import pint
ureg = pint.UnitRegistry()

In [ ]:
def calculate_enable_resisotrs(v_in: float, v_start: float, v_stop: float):
  i_p = 0.7 * ureg.uA
  i_h = 1.4 * ureg.uA
  v_en_fall = 1.17 * ureg.V
  v_en_rise = 1.21 * ureg.V

  r_1 = (v_start * (v_en_fall/v_en_rise) - v_stop) / (i_p * (1 - v_en_fall/v_en_rise) + i_h)
  r_2 = (r_1 * v_en_fall) / (v_stop - v_en_fall + r_1 * (i_p + i_h))
  v_en = (r_2 * v_in + r_1 * r_2 * (i_p + i_h)) / (r_1 + r_2)

  return {
      'r_1': r_1.to("kohm"),
      'r_2': r_2.to("kohm"),
      'v_en': v_en,
  }

def calculate_feedback_resistor(v_out: float, r_fbb: float):
  v_ref = 0.8 * ureg.V
  r_fbt = (v_out - v_ref) / v_ref * r_fbb
  return {'r_fbt': r_fbt}

def calculate_output_voltage(r_fbt: float, r_fbb: float):
  v_ref = 0.8 * ureg.V
  v_out = ((r_fbt / r_fbb) * v_ref) + v_ref

  return {'v_out': v_out.to("volt")}

def calculate_inductor(v_in: float, v_out: float, f_sw: float, i_out_max: float, k: float = 0.4):
  l = ((v_in - v_out)/(f_sw * k * i_out_max)) * (v_out / v_in)
  delta_i_l = (v_out / v_in) * ((v_in - v_out)/(l * f_sw))
  i_l_peak = i_out_max + (delta_i_l/2)
  i_l_rms = numpy.sqrt(i_out_max**2 + (delta_i_l**2/12))
  return {
      'l': l.to('uH'),
      'delta_i_l': delta_i_l,
      'i_l_peak': i_l_peak,
      'i_l_rms': i_l_rms,
  }

def calculate_output_capacitance(target_ripple: float, i_out: float, f_sw: float, k: float = 0.4):
  esr_ripple = target_ripple / (k * i_out)
  c_out_ripple = ((k * i_out)/(8 * f_sw * target_ripple))

  return {
    'esr_ripple': esr_ripple.to('mohm'),
    'c_out_ripple': c_out_ripple.to('uF'),
  }

def calculate_soft_start_capacitance(t_ss: float):
  v_ref = 0.8 * ureg.V
  i_ss = 5.5 * ureg.uA
  c_ss = (t_ss * i_ss) / v_ref

  return {
    'c_ss': c_ss.to('nF'),
  }


In [ ]:
calculate_enable_resisotrs(24 * ureg.V, 14 * ureg.V, 12 * ureg.V)

{'r_1': 1080.1393728222997 <Unit('kiloohm')>,
 'r_2': 96.48303765950824 <Unit('kiloohm')>,
 'v_en': 2.154 <Unit('volt')>}

In [ ]:
calculate_feedback_resistor(10 * ureg.V, 10 * ureg.kohm)

{'r_fbt': 114.99999999999999 <Unit('kiloohm')>}

In [ ]:
calculate_output_voltage(115 * ureg.kohm, 10 * ureg.kohm)

{'v_out': 10.000000000000002 <Unit('volt')>}

In [ ]:
115 * 0.01

1.1500000000000001

In [ ]:
calculate_inductor(24 * ureg.V, 10 * ureg.V, 500 * ureg.kHz, 3 * ureg.A)


{'l': 9.722222222222223 <Unit('microhenry')>,
 'delta_i_l': 1.2 <Unit('ampere')>,
 'i_l_peak': 3.6 <Unit('ampere')>,
 'i_l_rms': 3.0199337741082997 <Unit('ampere')>}

In [ ]:
calculate_inductor(30 * ureg.V, 5 * ureg.V, 500 * ureg.kHz, 3 * ureg.A)


{'l': 6.944444444444444 <Unit('microhenry')>,
 'delta_i_l': 1.2000000000000002 <Unit('ampere')>,
 'i_l_peak': 3.6 <Unit('ampere')>,
 'i_l_rms': 3.0199337741082997 <Unit('ampere')>}

In [ ]:
calculate_output_capacitance(target_ripple=30 * ureg.mV, i_out = 3 * ureg.A, f_sw = 500 * ureg.kHz)

{'esr_ripple': 24.999999999999996 <Unit('milliohm')>,
 'c_out_ripple': 10.0 <Unit('microfarad')>}

In [ ]:
calculate_soft_start_capacitance(t_ss = 15 * ureg.ms)

{'c_ss': 103.125 <Unit('nanofarad')>}